In [1]:
import cv2
import numpy as np
import math

In [2]:
labelsPath = "./coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

np.random.seed(42)

weightsPath = "./yolov3.weights"
configPath = "./yolov3.cfg"

net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
name = net.getLayerNames()
out = net.getUnconnectedOutLayers()
outputlayernames = [name[i[0] - 1] for i in out]

In [3]:
#Social Distance Tracking on a video footage
cap = cv2.VideoCapture("test_vid.mp4")
_, frame = cap.read()
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))

In [ ]:
while cap.isOpened():
    ret, img = cap.read()
    if ret == False:
        continue
    
    #Object Detection
    (H, W) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    output = net.forward(outputlayernames)
    boxes, confidences, class_ids = [], [], []
    height, width, shape = img.shape
    
    for m in output:
        for detection in m:
            scores = detection[5:]
            class_ids = np.argmax(scores)
            confidence = scores[class_ids]
            if confidence > 0.6 and class_ids == 0:
                print(detection[0:4])
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))

    #Boxes around the objects
    ind = []
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.5)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            ind.append([x, y, w, h])

    #Alert for not maintaining social distance
    red_boxes = []
    for j in range(len(ind)):
        for k in range(len(ind)):
            if k == j:
                break
            else:
                x1, y1 = ind[j][:2]
                x2, y2 = ind[k][:2]
                dist = np.sqrt(((x2 - x1) ** 2) + ((y2 - y1) **2))
                if dist <= 130:
                    red_boxes.append(ind[j])
                    red_boxes.append(ind[k])
    for r in red_boxes:        
        A, B, C, D = r
        cv2.putText(img, "ALERT", (int(A), int(B) - 5), cv2.FONT_HERSHEY_PLAIN, 0.5, [0, 0, 255], 2)
        cv2.rectangle(img, (A, B), (A + C, B + D), [0, 0, 255], 2)
    
    cv2.imshow('img',img)
    
    if cv2.waitKey(1) == ord("q"):
        break
    out.write(img)

cap.release()
out.release()

[0.7043243  0.06960125 0.072091   0.13086545]
[0.79708576 0.5558766  0.08935421 0.28575313]
[0.7043723  0.06954061 0.07247017 0.13067684]
[0.7992563  0.52485543 0.09510846 0.26711518]
[0.70327103 0.07101925 0.07286454 0.13343889]
[0.7983148  0.5155338  0.10634077 0.2927359 ]
[0.701261   0.0709777  0.07252021 0.13275234]
[0.8130007  0.49075568 0.08288166 0.279926  ]
[0.70185554 0.07001542 0.06421804 0.13470519]
[0.81594247 0.45199484 0.09001548 0.2891935 ]
[0.7008162  0.06938691 0.06158992 0.13356641]
[0.82892835 0.43268988 0.06749047 0.25895402]
[0.70175356 0.07088395 0.06029419 0.13744992]
[0.83425057 0.41113895 0.06551164 0.26553586]
[0.7028224  0.07251178 0.05499117 0.13885652]
[0.8398295 0.398108  0.0688464 0.2626991]
[0.70483077 0.07427819 0.04971451 0.14846049]
[0.8422376  0.36016142 0.06962323 0.25356942]
[0.7040679  0.07390893 0.04876459 0.15393299]
[0.85075176 0.33151454 0.06012195 0.22839454]
[0.70355904 0.07380483 0.0519789  0.15711215]
[0.8528538  0.31588304 0.06920398 0.22

[0.55614597 0.06879146 0.05295452 0.13580766]
[0.6700618  0.05796731 0.05972637 0.10785358]
[0.6450328  0.18638428 0.06631412 0.2131892 ]
[0.66997266 0.05215071 0.0353787  0.11508739]
[0.55473626 0.06792884 0.05187912 0.13357034]
[0.67040724 0.05813477 0.06201272 0.10533018]
[0.6472054  0.19744097 0.06669655 0.2210335 ]
[0.6690734  0.05352627 0.03688508 0.11598885]
[0.55360323 0.06869884 0.05146006 0.13728431]
[0.66783595 0.05868322 0.05932148 0.11003292]
[0.6465098  0.20006658 0.07051565 0.22121409]
[0.6675355  0.05300039 0.03535843 0.11277192]
[0.55107373 0.07016684 0.0495141  0.13649109]
[0.667197   0.05840414 0.05930317 0.11031508]
[0.64361614 0.2020536  0.06882998 0.2232505 ]
[0.6668012  0.05316454 0.03609192 0.11452076]
[0.55199575 0.06530294 0.03009491 0.16247734]
[0.5503954  0.06955852 0.04833337 0.13935958]
[0.666163   0.06003648 0.05794627 0.11375503]
[0.641471   0.20672204 0.06557135 0.21526927]
[0.665573   0.05409494 0.03300117 0.1161272 ]
[0.55071443 0.06977223 0.04618818 

[0.489498   0.12898567 0.03511674 0.17978162]
[0.60211444 0.11932485 0.03864327 0.18271855]
[0.56861913 0.34186894 0.05709048 0.22809997]
[0.14427191 0.83974594 0.08894729 0.2919681 ]
[0.48861992 0.13400899 0.03549766 0.18506463]
[0.6011542  0.12100684 0.04016955 0.18113106]
[0.56824917 0.35137218 0.05808743 0.24831495]
[0.14584507 0.8391585  0.08346158 0.3006108 ]
[0.48756295 0.13177826 0.03569688 0.18129025]
[0.59893525 0.12366144 0.04066812 0.1799519 ]
[0.56748587 0.35279858 0.05766005 0.24920729]
[0.14637922 0.8316206  0.08573242 0.31475708]
[0.48620364 0.13576557 0.03543873 0.18734516]
[0.5967049  0.12394505 0.0400776  0.17967817]
[0.5644995  0.35772943 0.06299918 0.23765434]
[0.14900675 0.81480587 0.08101476 0.2962106 ]
[0.48380902 0.14452416 0.03913209 0.19437562]
[0.5938353  0.12724258 0.04277195 0.18258199]
[0.56063783 0.37412217 0.06271224 0.23958412]
[0.15962762 0.8013621  0.08456016 0.30649275]
[0.48477447 0.14794895 0.03906016 0.2005843 ]
[0.5926739  0.12911342 0.04140404 

[0.5118756  0.22154167 0.04175694 0.21282686]
[0.4082012  0.26023751 0.04937963 0.22331983]
[0.32500762 0.5910521  0.05590264 0.24679019]
[0.4532861  0.62435466 0.0818387  0.3181341 ]
[0.5085853  0.2252375  0.0406506  0.21045081]
[0.4070539  0.26314875 0.05207649 0.22866692]
[0.328341   0.58932406 0.06066128 0.24622525]
[0.45324424 0.62932956 0.08097834 0.31813815]
[0.50803334 0.22712204 0.04225833 0.20734373]
[0.40585464 0.26329666 0.05286223 0.22477339]
[0.32983795 0.5953216  0.06356724 0.2566414 ]
[0.45189273 0.6345263  0.07982579 0.31103992]
[0.08875021 0.92767185 0.08387463 0.14026749]
[0.40138102 0.26299915 0.04918385 0.2154993 ]
[0.5052287  0.23718905 0.04701523 0.21159938]
[0.3358115  0.59125865 0.06158888 0.25801376]
[0.44181597 0.63532436 0.08407377 0.2834389 ]
[0.10010568 0.9192006  0.09632646 0.15714481]
[0.39754838 0.26293734 0.04405232 0.21245399]
[0.5044921  0.23897815 0.04842642 0.21111025]
[0.33887073 0.5627215  0.05123413 0.20865002]
[0.44152087 0.62814534 0.06747755 

[0.31933162 0.39109468 0.05772531 0.28201208]
[0.43605444 0.39123553 0.08533333 0.29979828]
[0.45239905 0.4509438  0.05451914 0.20433705]
[0.28418624 0.7227614  0.06668398 0.24357413]
[0.26105416 0.8806235  0.13193287 0.2240647 ]
[0.2554627  0.8907626  0.18737015 0.21273595]
[0.31613812 0.39480522 0.05589849 0.2813671 ]
[0.43438753 0.3921944  0.08943231 0.3005967 ]
[0.45657894 0.4473037  0.05421322 0.20172955]
[0.2880912  0.74607426 0.07686864 0.3013183 ]
[0.25617987 0.8798234  0.12462533 0.22944672]
[0.4196456  0.358062   0.06987453 0.21053392]
[0.3124188  0.40045974 0.05841663 0.2516217 ]
[0.46454424 0.44096822 0.05519273 0.21859114]
[0.2977767  0.7448132  0.07631499 0.30649588]
[0.25054067 0.8984582  0.13595434 0.19911113]
[0.41949293 0.36349455 0.06853973 0.21792   ]
[0.31088415 0.40342712 0.06119539 0.23784305]
[0.46605694 0.44006544 0.05386418 0.21776272]
[0.29948086 0.7396138  0.07873005 0.3008676 ]
[0.25035244 0.90261626 0.133538   0.19103622]
[0.41854906 0.36839992 0.06386183 

[0.52074397 0.0584951  0.06412317 0.1216649 ]
[0.52284104 0.34224686 0.04164318 0.20652875]
[0.19798087 0.5549706  0.06169362 0.25777307]
[0.33637652 0.5531256  0.09448417 0.27833727]
[0.41995367 0.587914   0.06418405 0.29600146]
[0.52724594 0.05081419 0.03920822 0.11408298]
[0.5202561  0.05866639 0.0685202  0.11811251]
[0.5230425  0.34117624 0.0408445  0.20209816]
[0.19460429 0.55917716 0.06026191 0.2658632 ]
[0.3533189  0.56063664 0.10835873 0.2836209 ]
[0.42670488 0.5703892  0.0566085  0.2768199 ]
[0.51925606 0.05897088 0.06432837 0.11800646]
[0.52563846 0.33983314 0.03953557 0.19929688]
[0.18861903 0.5632774  0.06374441 0.27000362]
[0.35418683 0.56429017 0.12002479 0.2839355 ]
[0.43121293 0.56677103 0.05652902 0.28652608]
[0.5250902  0.05252904 0.04119503 0.11595832]
[0.51499027 0.06048248 0.06142394 0.1207763 ]
[0.5263243  0.33851263 0.04145405 0.20045222]
[0.18399951 0.56106776 0.06237835 0.25111705]
[0.35483974 0.56560004 0.1312756  0.27726835]
[0.43414018 0.5621024  0.0566862  